In [12]:
# Instalação de Bibliotecas
!pip install -q google-genai google-adk scikit-learn

In [13]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [15]:
# Importações
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [17]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [18]:
# Funções Auxiliares (como no exemplo)
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [21]:
# Agente de IA 1
def agente_coletor_dados(dados):
    coletor = Agent(
        name="agente_coletor_dados",
        model=MODEL_ID,
        description="Agente que coleta dados de testes.",
        instruction="Você é um coletor de dados. Sua tarefa é organizar os dados dos testes em um formato adequado para análise.",
    )
    entrada = f"Dados dos testes: {dados}"
    return call_agent(coletor, entrada)

In [22]:
# Agente de IA 2
def agente_analista_dados(dados_coletados):
    analista = Agent(
        name="agente_analista_dados",
        model=MODEL_ID,
        description="Agente que analisa dados.",
        instruction="Você é um analista de dados. Analise os dados fornecidos para identificar padrões relevantes para a Síndrome de Down.",
    )
    entrada = f"Dados coletados: {dados_coletados}"
    return call_agent(analista, entrada)

In [23]:
# Agente de IA 3
def agente_avaliador_risco(analise):
    avaliador = Agent(
        name="agente_avaliador_risco",
        model=MODEL_ID,
        description="Agente que avalia o risco.",
        instruction="Você é um avaliador de risco. Com base na análise, determine o nível de risco de Síndrome de Down (baixo, médio, alto).",
    )
    entrada = f"Análise dos dados: {analise}"
    return call_agent(avaliador, entrada)

In [24]:
# Agente de IA 4
def agente_relator(avaliacao_risco):
    relator = Agent(
        name="agente_relator",
        model=MODEL_ID,
        description="Agente que gera relatórios.",
        instruction="Você é um gerador de relatórios. Crie um relatório claro e conciso com a avaliação de risco e recomendações.",
    )
    entrada = f"Avaliação de risco: {avaliacao_risco}"
    relatorio = call_agent(relator, entrada)
    # Adicionando a declaração importante
    declaracao = "\n\nIMPORTANTE: Este relatório é resultado de um desenvolvimento estudantil e pesquisa de desenvolvimento de software. Não deve ser considerado como laudo médico e não tem qualquer valor para iniciar ou modificar um tratamento. Recomenda-se a consulta com especialistas da área médica para avaliação e orientação adequadas."
    return relatorio + declaracao

In [25]:
# Função Principal
def avaliar_risco_sindrome_down(dados_testes):
    dados_coletados = agente_coletor_dados(dados_testes)
    analise = agente_analista_dados(dados_coletados)
    avaliacao_risco = agente_avaliador_risco(analise)
    relatorio = agente_relator(avaliacao_risco)
    return relatorio

In [27]:
def coletar_dados_usuario():
    """Coleta os dados do usuário com validação."""

    dados = {}

    try:
        dados["idade_mae"] = int(input("Digite a idade da mãe: "))
        if dados["idade_mae"] < 15 or dados["idade_mae"] > 50:  # Limite de idade razoável
            raise ValueError("Idade da mãe fora do intervalo esperado.")

        dados["translucencia_nucal"] = float(input("Digite a translucência nucal (em mm): "))
        if dados["translucencia_nucal"] < 0 or dados["translucencia_nucal"] > 10:  # Limites razoáveis
            raise ValueError("Translucência nucal fora do intervalo esperado.")

        dados["ossonasal_presente"] = input("O osso nasal está presente? (sim/não): ").lower() == "sim"

        dados["frequencia_cardiaca_fetal"] = int(input("Digite a frequência cardíaca fetal (bpm): "))
        if dados["frequencia_cardiaca_fetal"] < 100 or dados["frequencia_cardiaca_fetal"] > 200:  # Limites
            raise ValueError("Frequência cardíaca fetal fora do intervalo esperado.")

        dados["comprimento_cabeca_nadega"] = float(input("Digite o comprimento cabeça-nádega (em mm): "))
        if dados["comprimento_cabeca_nadega"] < 40 or dados["comprimento_cabeca_nadega"] > 84:
            raise ValueError("Comprimento cabeça-nádega fora do intervalo esperado.")

    except ValueError as e:
        print(f"Erro: {e}")
        return None  # Indica que a coleta falhou

    return dados

In [28]:
# Exemplo de Uso
print("Bem-vindo ao Sistema de Avaliação de Risco de Síndrome de Down")
dados_testes = coletar_dados_usuario()

if dados_testes:
    relatorio = avaliar_risco_sindrome_down(dados_testes)
    print("\nRelatório de Avaliação:")
    print(relatorio)
else:
    print("\nNão foi possível gerar o relatório devido a erros na entrada.")

Bem-vindo ao Sistema de Avaliação de Risco de Síndrome de Down
Digite a idade da mãe: 34
Digite a translucência nucal (em mm): 0.9
O osso nasal está presente? (sim/não): Sim
Digite a frequência cardíaca fetal (bpm): 120
Digite o comprimento cabeça-nádega (em mm): 60

Relatório de Avaliação:
## Relatório de Avaliação de Risco

**Data:** [Data Atual]

**Assunto:** Avaliação de Risco - Risco Baixo

**Resumo:**

Este relatório apresenta uma avaliação do risco identificado como sendo de nível "Baixo". A análise detalhada a seguir descreve os fatores que contribuem para essa classificação e fornece recomendações para manter o risco em níveis aceitáveis.

**Avaliação de Risco:**

*   **Nível de Risco:** Baixo

**Justificativa:**

A classificação de risco "Baixo" indica que a probabilidade de ocorrência do evento adverso é pequena e/ou o impacto potencial, caso ocorra, é limitado e de fácil gerenciamento.  [Seria ideal detalhar os fatores específicos que levaram à classificação de "baixo" aqui